In [31]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from alibi.explainers import AnchorTabular
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.feature_selection import SelectFromModel
import warnings
warnings.filterwarnings("ignore")

In [6]:
import sys
import os
sys.path.append(os.path.abspath('/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine'))

from utils.helper import *
failures = load_failures_data('../data/model_data/failures.csv')
components = failures['component'].unique()
component_data = load_all_component_data(components)
data_splits = prepare_all_data_for_training(component_data, "target_class")
models = load_all_models(components, "xgb")
selected_features_data = fit_and_select_features(models, data_splits)
models = retrain_models_on_selected_features(models, selected_features_data, data_splits)

In [7]:
for component in components:
    globals()[f"{component}_predit_fn"] = lambda x: models[component].predict(x)
    globals()[f"{component}_explainer"] = AnchorTabular(globals()[f"{component}_predit_fn"], selected_features_data[component][2])
    globals()[f"{component}_explainer"].fit(selected_features_data[component][0].to_numpy())


In [8]:
def time_convert(sec):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  return "Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec)

In [9]:
for component in components:
    globals()[f"{component}_test_np"] = selected_features_data[component][1].to_numpy()
    globals()[f"{component}_test_length"] = globals()[f"{component}_test_np"].shape[0]
    globals()[f"anchors_{component}"] =[ ]
    


In [ ]:
start_time = time.time()
for component in components:
    for i in range(globals()[f"{component}_test_length"]):
        explanation = globals()[f"{component}_explainer"].explain(globals()[f"{component}_test_np"][i],threshold=0.95)
        globals()[f"anchors_{component}"].append(explanation)
        current_time = time.time()
        time_lapsed = current_time - start_time
        print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, globals()[f"{component}_test_length"], time_convert(time_lapsed)), end="\r", flush=True)
    print(f"{component} done")
    print(time_convert(time.time() - start_time))

# Save Anchors as JSON

In [30]:
anchors_list = []
for component in components:
   for index , elem in enumerate(globals()[f"anchors_{component}"]):

    anchor = ' AND '.join(elem["data"]["anchor"])  # Assuming this is a list of strings
    precision = float(elem["data"]["precision"])
    coverage = float(elem["data"]["coverage"])
    anchor_data = {
        "id": index,
        "component": component,
        "anchor": anchor,
        "precision": round(precision, 2),
        "coverage": round(coverage, 2)
    }
    anchors_list.append(anchor_data)
import json
json_output = json.dumps(anchors_list, indent=4)
with open('anchors_output.json', 'w') as json_file:
    json.dump(anchors_list, json_file, indent=4)
